# Description of the `FinanceNewsScraper` Class

The `FinanceNewsScraper` class is designed to scrape financial news articles from the business section of Google News based on a set of specified buzzwords and a given date range.

- **Initialization (`__init__`)**: 
  - The scraper accepts two sets of buzzwords:
    - **Must-have buzzwords**: Keywords that must appear in the article title or description.
    - **Percentage-based buzzwords**: Keywords that need to match a certain percentage within the article.
  - It also takes a `start_date`, `end_date`, and an interval for scraping in chunks (e.g., weekly).

- **URL Construction (`construct_url`)**: 
  - This function builds a Google News RSS URL specifically for the business section, incorporating the provided buzzwords and date range.

- **Fetching Data (`fetch_rss_feed`)**: 
  - This function retrieves the RSS feed using the constructed URL, retrying up to three times if errors are encountered.
  - **Robust Retry Mechanism**:
      - To ensure stable scraping even when there are network issues, the class includes a retry mechanism. It retries the process multiple times if it fails to retrieve the Yahoo Finance page, adding reliability to the data extraction process.


- **Keyword Matching**:
  - **Must-have buzzwords**: Ensures that at least one must-have buzzword appears in the article's title or description.
  - **Percentage-based buzzwords**: Verifies that a minimum percentage of the provided buzzwords are present in the article.

- **Article Parsing (`parse_articles`)**: 
  - This function parses the RSS feed and extracts relevant information such as the article title, URL, and publication date, but only for articles that match the buzzword criteria.

- **Scraping (`scrape`)**: 
  - This method iterates through the specified date range, fetching and parsing articles in chunks as defined by the provided interval.

- **Saving to CSV (`save_to_csv`)**: 
  - After scraping, the articles are saved to a CSV file using the `pandas` library for easy storage and further analysis.

This class simplifies the process of scraping Google News for business-related articles based on keywords, while also offering functionality to save the results as a CSV file for later analysis.


In [114]:
import requests
import random
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import math
import time
import csv
import pandas as pd

In [458]:
class FinanceNewsScraper:
    def __init__(self, primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, interval):
        """
        Initialize the scraper with two sets of buzzwords, start date, end date, and required percentage.
        :param primary_buzzwords: List of buzzwords that must be present.
        :param secondary_buzzwords: List of buzzwords to search for with percentage matching.
        :param start_date: The start date (YYYY-MM-DD) for the articles.
        :param end_date: The end date (YYYY-MM-DD) for the articles.
        :param required_percentage: The percentage of percentage-based buzzwords that should be present (default 60%).
        """
        self.primary_buzzwords = primary_buzzwords
        self.secondary_buzzwords = secondary_buzzwords
        self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
        self.end_date = datetime.strptime(end_date, '%Y-%m-%d')
        self.required_percentage = required_percentage / 100  # Convert percentage to decimal for calculations
        self.base_url = "https://news.google.com/rss"
        self.interval = interval
        self.max_retries = 3  # Number of retries in case of failure

    def construct_url(self, start_date, end_date):
        """
        Construct the Google News RSS URL with all buzzwords and date range.
        :return: The constructed URL.
        """
        combined_buzzwords = self.primary_buzzwords + self.secondary_buzzwords
        query = " AND ".join(combined_buzzwords)  # Combine all buzzwords with 'AND' to ensure all words are present
        formatted_query = query.replace(" ", "%20")  # Format query for URL
        
        url = f"{self.base_url}?q={formatted_query}+after:{start_date}+before:{end_date}&hl=en-US&gl=US&ceid=US:en"
        
        return url

    def fetch_rss_feed(self, start_date, end_date, max_retries=5, backoff_factor=2):
        """
        Fetch the RSS feed from Google News for a given date range, with retries and exponential backoff to avoid 503 errors.
        :param start_date: The start date for fetching articles.
        :param end_date: The end date for fetching articles.
        :param max_retries: Maximum number of retries if the request fails.
        :param backoff_factor: Factor by which the wait time increases after each failure.
        :return: BeautifulSoup object with the RSS feed content.
        """
        rss_url = self.construct_url(start_date, end_date)
        attempt = 0
        delay = 5  # Start with an initial delay of 5 seconds

        while attempt < max_retries:
            try:
                response = requests.get(rss_url, headers={'User-Agent': 'Mozilla/5.0'})
                
                if response.status_code == 200:
                    return BeautifulSoup(response.content, 'xml')  # Parsing as XML
                else:
                    print(f"Failed to retrieve RSS feed with status code {response.status_code}. Retrying...")

            except requests.RequestException as e:
                print(f"Error fetching the RSS feed: {e}. Retrying...")

            # Apply the exponential backoff
            attempt += 1
            time.sleep(delay)
            delay *= backoff_factor  # Increase the delay exponentially

        print("Max retries exceeded. Could not fetch the RSS feed.")
        return None


    def contains_any_primary_buzzwords(self, text):
        """
        Check if any must-have buzzwords are present in the given text.
        :param text: The text to search for must-have buzzwords (case-insensitive).
        :return: True if at least one must-have buzzword is found, False otherwise.
        """
        text = text.lower()
        return any(buzzword.lower() in text for buzzword in self.primary_buzzwords)

    def contains_percentage_of_buzzwords(self, text):
        """
        Check if at least the required percentage of percentage-based buzzwords are present in the given text.
        :param text: The text to search for percentage-based buzzwords (case-insensitive).
        :return: True if the required percentage of percentage-based buzzwords are found, False otherwise.
        """
        text = text.lower()
        buzzwords_found = sum(1 for buzzword in self.secondary_buzzwords if buzzword.lower() in text)
        required_count = math.ceil(len(self.secondary_buzzwords) * self.required_percentage)
        
        # The condition now checks if at least the required count of buzzwords is found
        return buzzwords_found >= required_count

    def parse_articles(self, soup):
        """
        Parse the RSS feed and extract article information.
        Only return articles where all must-have buzzwords and a percentage of percentage-based buzzwords are found.
        :param soup: BeautifulSoup object of the RSS feed.
        :return: List of dictionaries with article titles, URLs, and publication dates.
        """
        articles = []
        for item in soup.find_all('item'):
            title = item.title.text
            link = item.link.text
            description = item.description.text if item.description else ""
            pub_date = item.pubDate.text
            pub_date = datetime.strptime(pub_date, '%a, %d %b %Y %H:%M:%S %Z')  # Format the date
            
            # Check if any must-have buzzwords are present in title or description
            first_100_words = " ".join(description.split()[:100])
            if self.contains_any_primary_buzzwords(title) or self.contains_any_primary_buzzwords(first_100_words):
                # Check if the required percentage of percentage-based buzzwords are present
                if self.contains_percentage_of_buzzwords(title) or self.contains_percentage_of_buzzwords(first_100_words):
                    articles.append({'title': title, 'url': link, 'date': pub_date})
        return articles

    def scrape(self):
        """
        Scrape the RSS feed and extract articles that match both must-have and percentage-based buzzwords.
        :return: List of articles (titles, URLs, and dates).
        """
        all_articles = []
        delta = timedelta(days=self.interval)  # Fetch in intervals (e.g., weekly)
        current_start_date = self.start_date
        print(f"Fetching articles from {current_start_date.strftime('%Y-%m-%d')} to {self.end_date.strftime('%Y-%m-%d')}")

        # Loop through the date range with the specified interval
        while current_start_date < self.end_date:
            current_end_date = min(current_start_date + delta, self.end_date)

            soup = self.fetch_rss_feed(current_start_date.strftime('%Y-%m-%d'),
                                       current_end_date.strftime('%Y-%m-%d'))
            if soup:
                articles = self.parse_articles(soup)
                all_articles.extend(articles)

            current_start_date += delta  # Move to the next interval

        if all_articles:
            print(f"Found {len(all_articles)} articles matching the criteria.")
        else:
            print("No articles found matching the criteria.")
        return all_articles

    def save_to_csv(articles, filename):
        """
        Save the scraped articles to a CSV file using pandas.
        :param articles: List of articles with title, URL, and date.
        :param filename: The name of the CSV file (default is "articles.csv").
        """
        # Convert the list of articles to a pandas DataFrame and drop duplicates in case there are any
        df = pd.DataFrame(articles).drop_duplicates()
        
        # Save DataFrame to CSV
        df.to_csv(filename, index=False, encoding='utf-8') 

In [459]:
primary_buzzwords = ["gas","petrol","fuel","price","oil"]
secondary_buzzwords = ["israel", "gaza", "palestine", "conflict", "war", "hamas", 
                       "ukraine", "russia"]  # List of buzzwords to search for
start_date = '2024-10-21'
end_date = '2024-10-22'  # End date
required_percentage = 6  # required_percentage% of the secondary buzzwords should be in title or description
interval = 1

scraper_news = FinanceNewsScraper(primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, interval)
articles_news = scraper_news.scrape()

# Save the output as CSV files 
FinanceNewsScraper.save_to_csv(articles_news, "google_news_geopolitical_energy.csv")

Fetching articles from 2024-10-21 to 2024-10-22
No articles found matching the criteria.


### `FinanceNewsAPIScraper` Class Description

The `FinanceNewsAPIScraper` class is designed to fetch, filter, and save news articles from NewsAPI based on specified buzzwords. Key functionalities:

- **Initialization**: Takes in an API key, primary and secondary buzzwords, date range, and retry settings.
- **Fetch News**: Sends API requests and retries if rate limits are hit.
- **Filter**: Filters articles to ensure primary buzzwords are present, with a required percentage of secondary buzzwords.
- **Display & Save**: Displays the filtered articles and provides an option to save them to a CSV file.

### Key Methods:
- `fetch_news()`
- `filter_articles()`
- `display_articles()`


In [460]:
class FinanceNewsAPIScraper:
    def __init__(self, api_key, primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after):
        self.api_key = api_key
        self.primary_buzzwords = primary_buzzwords
        self.secondary_buzzwords = secondary_buzzwords
        self.start_date = start_date
        self.end_date = end_date
        self.required_percentage = required_percentage / 100
        self.base_url = 'https://newsapi.org/v2/everything'
        self.retry_after = retry_after

    def contains_any_primary_buzzwords(self, text):
        text = text.lower()
        return any(buzzword.lower() in text for buzzword in self.primary_buzzwords)

    def contains_required_percentage_of_secondary_buzzwords(self, text):
        text = text.lower()
        buzzwords_found = sum(1 for buzzword in self.secondary_buzzwords if buzzword.lower() in text)
        required_count = math.ceil(len(self.secondary_buzzwords) * self.required_percentage)
        return buzzwords_found >= required_count

    def fetch_news(self, retries=3):
        params = {
                'q': ' OR '.join(self.primary_buzzwords + self.secondary_buzzwords),
                'apiKey': self.api_key,
                'from': self.start_date,
                'to': self.end_date,
                'language': 'en',
                'sortBy': 'relevancy'
            }


        attempt = 0
        while attempt < retries:
            response = requests.get(self.base_url, params=params)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print(f"Rate limit exceeded. Retrying after {self.retry_after} seconds...")
                time.sleep(self.retry_after)
            else:
                print(f"Failed to fetch news articles. Status code: {response.status_code}")
                return None
            attempt += 1
        print("Max retries exceeded. Could not fetch the news.")
        return None

    def filter_articles(self, news_data):
        filtered_articles = []
        if news_data and 'articles' in news_data:
            for article in news_data['articles']:
                title = article['title']
                description = article['description'] or ""
                content = title + " " + description
                if self.contains_any_primary_buzzwords(content) and self.contains_required_percentage_of_secondary_buzzwords(content):
                    filtered_articles.append(article)
        return filtered_articles

    def display_articles(self, articles):
        if articles:
            for i, article in enumerate(articles, start=1):
                print(f"{i}. {article['title']} ({article['publishedAt']})")
        else:
            print("No articles found.")

    def scrape_and_filter_news(self):
        news_data = self.fetch_news()
        if news_data:
            filtered_articles = self.filter_articles(news_data)
            self.display_articles(filtered_articles)
            return filtered_articles  # Ensure filtered_articles is returned
        else:
            print("Failed to retrieve or filter articles.")
            return []

    def save_to_csv(self, articles, filename):
        if articles:
            data = [{
                'title': article['title'],
                'publishedAt': article['publishedAt'],
                'url': article['url']
            } for article in articles]
            df = pd.DataFrame(data)
            df.to_csv(filename, index=False, encoding='utf-8')
            print(f"Articles saved to {filename}")
        else:
            print("No articles to save.")


In [385]:
api_keys = ['51f3c8bce6b1473e9537d03fe37815e3','5d7f3433c9404b6aaba5c5db771f2c79','25d106c70b3c4ff3af1fb174e0afc2ed']

**News Focus:** `Geopoliticial conflicts`


In [ ]:
secondary_buzzwords = ["israel", "gaza", "palestine", "conflict", "war", "hamas", 
                       "ukraine", "russia","airstrike","attack", "crisis","oil","prices","nato","invasion"
                       "iran","afghanistan","china","taiwan","military",
                       "indo-pacific","south china sea","market","nuclear","escalate","zelensky","putin"]  # List of buzzwords to search for
required_percentage = 6
retry_after = 60
start_date = '2024-09-23'
end_date = '2024-10-22'

**News Focus:** `Climate events`


In [461]:
secondary_buzzwords = ["extreme weather", "climate crisis", "rising sea levels", "coastal erosion", "heatwaves",
    "atmospheric rivers", "carbon emissions","greenhouse gases", "renewable energy", "deforestation", "carbon footprint",
    "sustainable development", "biodiversity loss","hurricane","tsunami","floods"]
 # List of buzzwords to search for
required_percentage = 6
retry_after = 60
start_date = '2024-09-23'
end_date = '2024-10-22'

**News Focus:** `Elections`

In [571]:
secondary_buzzwords = ["trump","kamala","harris","biden","modi","narendra","xi","putin", "election", 
                       "2024","campaign","vote","voter"]
 # List of buzzwords to search for
required_percentage = 6
retry_after = 60
start_date = '2024-09-23'
end_date = '2024-10-22'

**News Focus:** `Pandemics`

In [577]:
secondary_buzzwords = ["pandemic","epidemic","covid","corona","covid","bird-flu", "public health crisis", "outbreak", "quarantine",
                        "lockdown","infection", "epicenter","virus","bacteria","vaccine","epicenter"]
 # List of buzzwords to search for
required_percentage = 6
retry_after = 60
start_date = '2024-09-23'
end_date = '2024-10-22'

**News Focus:** `Government Policies`

In [663]:
secondary_buzzwords = ["tariffs", "sanctions", "subsidies", "regulations", "reforms","taxation", "immigration","trade",
                         "defense", "cybersecurity","privacy", "surveillance", "government", "monetary",
                        "labor", "manufacturing", "exports", "imports", "automation","health", "care","housing"] # List of buzzwords to search for
               
required_percentage = 6
retry_after = 60
start_date = '2024-09-23'
end_date = '2024-10-22'

**Industry Focus:** `Energy`

**Timeline:** `One month`

In the US the two biggest stocks in these industries are `ExxonMobil` (XOM) and `Chevron` (CVX)

*Stock Focus*: `XOM`, `CVX`, `WFRD`, `VNOM`.

In [633]:
primary_buzzwords = ["exxon", "xom", "chevron", "cvx", "viper", "vnom", "murphy", "mur","gas",
                     "petrol","fuel","price","oil","shell","carbon","hydrogen","nuclear","coal","fossil",
                     "energy","smart grid","power"]
scraper = FinanceNewsAPIScraper(api_keys[1], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_energy_one_month_govt.csv")

1. Mitch McConnell goes after Trump's trade policy: 'I'm not a tariff fan' (2024-09-24T21:06:42Z)
Articles saved to news_api_energy_one_month_govt.csv


**Industry Focus:** `Materials sector`

**Timeline:** `One month`

In the US the two biggest stocks in this industry are `Sherwin-Williams` (SHW) and `DuPont` (DD)

*Stock Focus*: `SHW`, `DD`, `GOLD`, `RIO`.

In [662]:
api_key = '5d7f3433c9404b6aaba5c5db771f2c79'  
primary_buzzwords = ["sherwin", "williams", "shw", "dupont", "barrick", "gold", "rio","tinto","steel",
                     "copper","aluminum","chemicals","minning","recycling","bio-based","materials","carbon",
                     "building","cement"]
scraper = FinanceNewsAPIScraper(api_keys[1], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_materials_one_month_govt.csv")

1. Titan Submersible Hearings Spotlight Multiple Issues With Its Carbon Fiber Hull (2024-09-25T21:01:25Z)
2. US bans new types of goods from China over allegations of forced labor (2024-10-02T18:46:00Z)
3. ICE's $2 Million Contract With a Spyware Vendor Is Under White House Review (2024-10-21T19:03:22Z)
4. 90-ton 3D printer to build world's biggest rocket (2024-10-22T18:00:00Z)
5. Foxconn Building Nvidia Superchip Facility In Mexico (2024-10-09T01:25:00Z)
Articles saved to news_api_materials_one_month_govt.csv


**Industry Focus:** `Industrials sector`

**Timeline:** `One month`

In the US the two biggest stocks in this industry are `UPS` (UPS) and `Raytheon` (RTX)

*Stock Focus*: `UPS`, `RTX`, `DAL`, `UAL`,`LMT`,`BA`.

In [657]:
primary_buzzwords = ["ups", "raytheon", "rtx", "delta","airlines","dal","united","ual","lmt","ba","boeing","machinery"
                     ,"engineering","automation","defense","aerospace"]
scraper = FinanceNewsAPIScraper(api_keys[1], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_industrials_one_month_govt.csv")

1. DJI sues the US Department of Defense for labeling it a ‘Chinese Military Company’ (2024-10-19T07:28:04Z)
2. The West is trying to starve the wrong part of Russia's war machine, defense experts say (2024-10-20T11:23:01Z)
3. Russia's defense industry has a massive labor shortage. Rules on what jobs women can do are part of the issue. (2024-10-05T10:43:02Z)
4. Boeing suppliers fear long-term jobs hit from strike (2024-09-30T10:04:34Z)
5. US laments 'disappointing' Swiss decision not to fully adopt latest EU sanctions against Russia (2024-10-18T16:45:28Z)
6. Putin hosts a summit in a bid to show the West it can't keep Russia off the global stage (2024-10-21T05:55:04Z)
7. Fears of automation are at the heart of the dockworkers strike (2024-10-02T14:22:50Z)
8. Mindustry: Automation Tower Defense RTS with Deep Strategy and Modding (2024-09-29T17:01:32Z)
9. Would Trump's tariffs trigger a global trade war? Experts weigh in. (2024-10-18T12:21:30Z)
10. Mark Cuban says Trump's call for 200% t

**Industry Focus:** `Utilities sector`

**Timeline:** `One month`

In the US the two biggest stocks in this industry are `Duke energy` (DUK) and `Consolidated Edison` (ED)

*Stock Focus*: `DUK`, `ED`.

In [655]:
primary_buzzwords = ["duke", "energy", "duk", "consolidated", "edison", "ed", "electricity",
                     "water supply","solar","wind","hydropower","energy prices","climate change","public utilities"]
scraper = FinanceNewsAPIScraper(api_keys[1], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_utilities_one_month_govt.vcsv")

1. US to unveil first of two decisions on more solar tariffs (2024-09-30T10:25:36Z)
2. DJI sues the US Department of Defense for labeling it a ‘Chinese Military Company’ (2024-10-19T07:28:04Z)
3. Mitch McConnell goes after Trump's trade policy: 'I'm not a tariff fan' (2024-09-24T21:06:42Z)
4. Opinion: How to design a US data privacy law (2024-09-28T11:15:11Z)
5. Trump is threatening tariffs on yet another business — and economists say it makes no sense (2024-09-25T08:00:02Z)
6. Would Trump's tariffs trigger a global trade war? Experts weigh in. (2024-10-18T12:21:30Z)
7. Scottish government: Assisted dying bill not in Holyrood's powers (2024-10-01T09:49:49Z)
8. India Plans Laptop Import Curbs To Boost Local Manufacturing (2024-10-18T14:03:00Z)
9. Britain targets Russian LNG sector with fresh shipping sanctions (2024-09-26T10:19:27Z)
10. Hurricanes Helene’s Floods Swamped a Hospital, Highlighting Climate Threats to Health Care (2024-10-02T18:45:00Z)
Articles saved to news_api_utilities_o

**Industry Focus:** `Healthcare sector`

**Timeline:** `One month`

In the US the two biggest stocks in this industry are `UnitedHealth Group` (UNH) and `Johnson & Johnson` (JNJ)

*Stock Focus*: `UNH`, `JNJ`.

In [653]:
primary_buzzwords = ["united health group", "health", "unh", "johnson & johnson","jnj","pharmaceuticals"
                     ,"biotechnology","drug", "clinical","fda"]
scraper = FinanceNewsAPIScraper(api_keys[0], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_health_one_month_govt.csv")

1. Huntsville Hospital Health System terminates contract with United Healthcare (2024-10-10T19:29:22Z)
2. FTC Sues Prescription Drug Middlemen for Artificially Inflating Insulin Prices (2024-09-23T10:58:56Z)
3. Why conventional wisdom on health care is wrong (a primer) (2020) (2024-10-17T15:57:54Z)
4. We left the US and moved to Mexico when I got pregnant 4 years ago. It's more affordable and our family is thriving. (2024-10-01T13:10:02Z)
5. Why is US health care like this? (2024-10-01T20:40:32Z)
6. Hurricanes Helene’s Floods Swamped a Hospital, Highlighting Climate Threats to Health Care (2024-10-02T18:45:00Z)
7. Will AI tools revolutionize public health? Not if they continue following old patterns, researchers argue (2024-10-09T00:14:19Z)
8. Novo Nordisk CEO blames drug 'middlemen' for high price of weight loss meds (2024-09-24T13:30:00Z)
Articles saved to news_api_health_one_month_govt.csv


**Industry Focus:** `Financials sector`

**Timeline:** `One month`

In the US the two biggest stocks in this industry are `Berkshire Hathaway` (BRK.A and BRK.B) and `JPMorgan Chase` (JPM)

*Stock Focus*: `BRK.A`, `BRK.BH`,`JPM`,`BAC`,`USB`, `pypl`.

In [652]:
primary_buzzwords = ["berkshire","hathaway","jp morgan","j.p. morgan","chase","bank","of america",
                     "brk.a","brk.b","bac","usb","paypal","pypl","stock","interest rates","inflation",
                     "bonds","capital","investment"]
scraper = FinanceNewsAPIScraper(api_keys[1], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_finance_one_month_govt.csv")

1. US economy grew at a solid 3% rate last quarter, government says in final estimate (2024-09-26T12:48:44Z)
Articles saved to news_api_finance_one_month_govt.csv


**Industry Focus:** `Consumers sector`

**Timeline:** `One month`

In the US the biggest stocks in this industry are `Coca-Cola` (KO), `Procter & Gamble` (PG), `Amazon` (AMZN), `Tesla` (TSLA) and McDonald's` (MCD)

*Stock Focus*: `KO`, `PG`, `AMZN`, `MCD`, `TSLA`,`TM`

In [651]:
primary_buzzwords = ["coca-cola","ko","procter & gamble","pg","amazon","macdonald's","amzn"
                     ,"mcd","retail","supply chain","household"
                     ,"automobile","tesla","tsla","toyota","tm","vehicles","car"
                     ,"ev","electric"]
scraper = FinanceNewsAPIScraper(api_keys[1], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_consumers_one_month_govt.csv")

1. DJI sues the US Department of Defense for labeling it a ‘Chinese Military Company’ (2024-10-19T07:28:04Z)
2. EU hits China with tariffs in electric car sales battle (2024-10-04T09:43:35Z)
3. Findlay calls on Swinney to ditch National Care Service plans (2024-10-03T12:45:20Z)
4. Scottish government: Assisted dying bill not in Holyrood's powers (2024-10-01T09:49:49Z)
5. Opinion: How to design a US data privacy law (2024-09-28T11:15:11Z)
6. 'Room for negotiation' on national care service plan - Gray (2024-09-29T10:51:54Z)
Articles saved to news_api_consumers_one_month_govt.csv


**Industry Focus:** `IT sector`

**Timeline:** `One month`

In the US the biggest stocks in this industry are `Apple` (AAPL), and `Microsoft` (MSFT)
*Stock Focus*: `AAPL`, `MSFT`, `QCOM`, `NVDA`, `CRWD`

In [650]:
primary_buzzwords = ["crowd strike holdings","crwd","apple","aapl","microsoft","msft",
                     "qualcomm","qcom", "nvidia","nvda", "it","information","technology","cybersecurity"
                     ,"semiconductors","ai","artificial"]
scraper = FinanceNewsAPIScraper(api_keys[1], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_it_one_month_govt.csv")

No articles found.
No articles to save.


**Industry Focus:** `Communication services sector`

**Timeline:** `One month`

In the US the biggest stocks in this industry are `Facebook` (FB), and `Alphabet` (GOOG)
*Stock Focus*: `FB`, `GOOG`, `WBD`, `NFLX`

In [647]:
primary_buzzwords = ["facebook","meta","fb","alphabet","goog","warner bros","wbd","netflix","nflx"
                     ,"broadband","5g","media","ott","television","streaming","platforms","film"
                     ,"movie","industry"]
scraper = FinanceNewsAPIScraper(api_keys[1], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_communication_one_month_govt.csv")

1. Findlay calls on Swinney to ditch National Care Service plans (2024-10-03T12:45:20Z)
2. Russia's defense industry has a massive labor shortage. Rules on what jobs women can do are part of the issue. (2024-10-05T10:43:02Z)
3. Scottish government: Assisted dying bill not in Holyrood's powers (2024-10-01T09:49:49Z)
Articles saved to news_api_communication_one_month_govt.csv


**Industry Focus:** `Real estate sector`

**Timeline:** `One month`

In the US the biggest stocks in this industry are `American Tower` (AMT), and `Simon Property Group` (SPG)
*Stock Focus*: `AMT`, `SPG`

In [649]:
primary_buzzwords = ["american tower","amt","simon property group","spg","real","estate","commercial"
                     ,"residential","property","housing","rental","home","prices","mortgage","land"
                     "bubble","loan","urban","planning"]
scraper = FinanceNewsAPIScraper(api_keys[1], primary_buzzwords, secondary_buzzwords, start_date, end_date, required_percentage, retry_after)
filtered_articles = scraper.scrape_and_filter_news()
scraper.save_to_csv(filtered_articles, "news_api_realestate_one_month_govt.csv")

1. Is the Dockworkers Union a Corporate Tool? (2024-10-04T22:30:01Z)
2. How Racist Policies Destroyed Public Housing and Created the American Suburbs (2024-09-25T08:55:27Z)
Articles saved to news_api_realestate_one_month_govt.csv
